In [14]:
using gmsh
using GR 
using LinearAlgebra
using SparseArrays 
using Plots
using LaTeXStrings

using LinearAlgebra
using SparseArrays 
using StructArrays
using StaticArrays
using StaticRanges
using FastGaussQuadrature

using IterativeSolvers
using Preconditioners

using BenchmarkTools
using Profile
using ProfileView

using Plots 

In [15]:
#include("C:/Users/IO/Downloads/gmsh-4.11.1-Windows64-sdk/gmsh-4.11.1-Windows64-sdk/lib/gmsh.jl")
using LinearAlgebra
struct Element
    p1::Array{Float64,1}
    p2::Array{Float64,1}
    p3::Array{Float64,1}
    p4::Array{Float64,1}
    n1::Int64
    n2::Int64
    n3::Int64
    n4::Int64
    vol::Float64
end

struct Mesh
    nnodes::Int64
    nelements::Int64 
    # specify one-dimensional array of elements as an array of structs. 
    # we worry about using structArray (if as all) later. 
    Elements::Array{Element,1}
    bndNodeIds::Vector{Int64}
    dofPerElement::Int64       
end 

In [16]:
function volume_tetrahedron(x1::Float64,x2::Float64,x3::Float64,x4::Float64,y1::Float64,y2::Float64,y3::Float64,y4::Float64,z1::Float64,z2::Float64,z3::Float64,z4::Float64)::Float64
    mat = [[x1 y1 z1 1]; [x2 y2 z2 1]; [x3 y3 z3 1]; [x4 y4 z4 1]];
    volume = abs(det(mat))/6;
    return volume
end

volume_tetrahedron (generic function with 1 method)

In [17]:
#generates the mesh for a cube 1*1*1

gmsh.initialize()
model = gmsh.model
model.add("cube_3D")

cl = 0.05 #mesh size

# Create a new model
gmsh.model.geo.addPoint(0, 0, 0, cl, 1)
gmsh.model.geo.addPoint(1, 0, 0, cl, 2)
gmsh.model.geo.addPoint(1, 1, 0, cl, 3)
gmsh.model.geo.addPoint(0, 1, 0, cl, 4)
gmsh.model.geo.addPoint(0, 0, 1, cl, 5)
gmsh.model.geo.addPoint(1, 0, 1, cl, 6)
gmsh.model.geo.addPoint(1, 1, 1, cl, 7)
gmsh.model.geo.addPoint(0, 1, 1, cl, 8)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 5, 8)
gmsh.model.geo.addLine(1, 5, 9)
gmsh.model.geo.addLine(2, 6, 10)
gmsh.model.geo.addLine(3, 7, 11)
gmsh.model.geo.addLine(4, 8, 12)

gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
gmsh.model.geo.addCurveLoop([5, 6, 7, 8], 2)
gmsh.model.geo.addCurveLoop([1,10,-5,-9], 3)
gmsh.model.geo.addCurveLoop([-2, 10, 6, -11], 4)
gmsh.model.geo.addCurveLoop([-3, 11, 7, -12], 5)
gmsh.model.geo.addCurveLoop([4, 9, -8, -12], 6)

for i in 1:6

    gmsh.model.geo.addPlaneSurface([i],i)

end

gmsh.model.geo.addSurfaceLoop([1,2,3,4,5,6], 1)
gmsh.model.geo.addVolume([1],1)

# Generate the mesh
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)

# Save the mesh to a file
gmsh.write("cube_3D.msh")

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00105596s, CPU 0s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.217525s, CPU 0.0

In [18]:
function genMesh(filename::String)::Mesh
    #the function takes the name of the file as input
    dofPerElement = 16;
    
    gmsh.initialize()
    gmsh.open(filename)
    
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)

    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end] node_coord[3:3:end]]; 
    sorted = sortslices(tosort , dims = 1);

    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    znode = sorted[:,4]

    #get the mesh elements
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(3)
    nelements = length(element_ids[1])
    #boundary elements
    bnd_types, bnd_el_ids, bnd_element_connectivity = gmsh.model.mesh.getElements(2)
    
    bnd_node_ids = [];
    #print(bnd_el_ids)
    for i in 1:length(bnd_el_ids[1])
        #retrieve global numbering of the local nodes of the current BOUNDARY element
        bnd_node1_id = bnd_element_connectivity[1][3*(i-1)+1]
        bnd_node2_id = bnd_element_connectivity[1][3*(i-1)+2]
        bnd_node3_id = bnd_element_connectivity[1][3*(i-1)+3]
        if !(bnd_node1_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node1_id)
        end
        if !(bnd_node2_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node2_id)
        end
        if !(bnd_node3_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node3_id)
        end
        
    end
    bnd_node_ids = sort(bnd_node_ids)
    gmsh.finalize()
    
    Elements = Array{Element,1}(undef,nelements);
    total_volume = 0;
    for i in 1:nelements
        
        #....retrieve global numbering of the local nodes of the current element
        node1_id = element_connectivity[1][4*(i-1)+1]
        node2_id = element_connectivity[1][4*(i-1)+2]
        node3_id = element_connectivity[1][4*(i-1)+3]
        node4_id = element_connectivity[1][4*(i-1)+4]
        
        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id]; xnode4 = xnode[node4_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id]; ynode4 = ynode[node4_id];
        znode1 = znode[node1_id]; znode2 = znode[node2_id]; znode3 = znode[node3_id]; znode4 = znode[node4_id];
        
        #calculate volume of the element and add it to the total volume
        vol = volume_tetrahedron(xnode1, xnode2, xnode3, xnode4, ynode1, ynode2, ynode3, ynode4, znode1, znode2, znode3, znode4)
        total_volume = total_volume + vol;
        #save nodes coordinates, nodes IDs and the volume of one element
        Elements[i] = Element([xnode1; ynode1; znode1],[xnode2; ynode2; znode2],[xnode3; ynode3; znode3],[xnode4; ynode4; znode4], node1_id, node2_id, node3_id, node4_id, vol)
    end
    #print(total_volume) #using cube_3D.msh the volume should be 1
    mesh = Mesh(nnodes, nelements, Elements, bnd_node_ids, dofPerElement)
    return mesh
end

genMesh (generic function with 1 method)

In [19]:
mesh = genMesh("cube_3D.msh")

Info    : Reading 'cube_3D.msh'...
Info    : 27 entities
Info    : 7350 nodes
Info    : 42579 elements
Info    : Done reading 'cube_3D.msh'


Mesh(7350, 36663, Element[Element([0.8687942768945374, 0.8687680307455696, 0.4811035662856836], [0.791557569141426, 0.9120054046759427, 0.495468358824924], [0.8101592480905414, 0.8251762089818118, 0.4943014014147774], [0.8017810754920548, 0.8834666216097109, 0.5447333791204171], 2987, 3492, 5304, 5699, 4.965832319952344e-5), Element([0.8270359952423089, 0.3606123119874011, 0.5954677675673312], [0.8313013086408507, 0.4214997986674724, 0.6475323656690691], [0.8015308851920228, 0.4492284026215273, 0.5783872449551982], [0.7726445817501689, 0.3886384128504871, 0.6289580527008207], 2897, 3762, 3476, 4370, 5.6168012073166355e-5), Element([1.0, 0.5999999999987025, 0.3071796769739583], [0.9122209320888198, 0.6000955970648635, 0.278367356506408], [0.9308000047881507, 0.5607185443286787, 0.335896283929552], [0.9381823163803846, 0.6480134186818157, 0.3076154558255481], 1616, 3435, 4195, 5108, 4.800973416254641e-5), Element([0.7412575348654493, 0.5555539904759468, 0.1208633738744909], [0.7504901594

In [20]:
function genLocStiffMat(element::Element)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4; 
    e1 = element.n1; e2 = element.n2; e3 = element.n3; e4 = element.n4; 
    vol = element.vol
    Iloc = SVector(e1, e1, e1, e1, e2, e2, e2, e2, e3, e3, e3, e3, e4, e4, e4, e4)
    Jloc = SVector(e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4)
    J = SMatrix{3,3}(p2[1] - p1[1], p2[2] - p1[2], p2[3] - p1[3],p3[1] - p1[1], p3[2] - p1[2], p3[3] - p1[3],p4[1] - p1[1], p4[2] - p1[2], p4[3] - p1[3])
    psi = SMatrix{3,4}(-1,1,0,0,-1,0,1,0,-1,0,0,1)
    m = MMatrix{3,4}(J\psi)
    x = transpose(m)*m*abs(det(J))/6
    Aloc = [x[1,:] ; x[2,:] ; x[3,:]; x[4,:] ] 
    #display(x)
    return Iloc, Jloc, Aloc
end

function genStiffMat(mesh)

    nelements = mesh.nelements
    dofPerElem = mesh.dofPerElement

    Avalues = zeros(Float64,dofPerElem*nelements)
    I = zeros(Int64,length(Avalues))
    J = zeros(Int64,length(Avalues)) 
    vol = 0

    for i = 1:nelements
        element = mesh.Elements[i]
        #vol += genLocStiffMat(element)
        Iloc, Jloc, Aloc = genLocStiffMat(element)
        irng             = mrange(dofPerElem*i-dofPerElem+1, dofPerElem*i) 
        I[irng]          = Iloc 
        J[irng]          = Jloc 
        Avalues[irng]    = Aloc  
    end
    #display(vol)
    A = sparse(I,J,Avalues)
    return A

end

S = genStiffMat(mesh)

7350×7350 SparseMatrixCSC{Float64, Int64} with 101042 stored entries:
⣿⣿⣘⣻⠀⠶⡇⢽⣿⢸⢾⠀⣶⡇⢶⡇⠀⠀⠀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣠⣂⣢⠠⣗⣀⣶⣶⣴⣌⣓
⣶⣸⣿⣿⡀⣀⡂⢒⣐⢐⣰⠀⣲⡆⢒⡆⠀⠀⢨⣿⣿⣿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣿⣻⣿⣿⣗⣿⣿⣿⣫
⢠⡄⠀⢨⣿⣿⡆⢠⣠⠠⢤⠀⣄⡄⢠⠄⠀⠀⢰⣽⣿⣻⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣯
⣍⣍⢨⢈⠈⣉⣿⣿⣯⢠⣩⠀⠀⡅⣨⡁⠀⠀⠨⣿⣿⣿⣾⣿⣿⣿⣿⣷⣿⣿⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣛⣛⢐⢘⠀⡚⠋⣛⢻⣶⣿⠀⣂⡃⠉⠁⠀⠈⢸⣿⣿⣿⣷⣯⣿⣿⣿⣿⣿⣿⣿⣯⣿⣿⣿⢿⣿⣿⣿⣿
⠚⠓⠐⠚⠀⠓⠃⠚⠛⠛⢻⣶⣿⡃⠒⠂⠀⠀⣘⢿⣿⣿⣿⣧⣿⣯⣿⣭⣿⣿⣿⣿⣿⣿⣿⣿⢿⣿⣻⣿
⠼⠿⠸⠾⠀⠽⠄⠤⠬⠸⠿⠻⠿⣧⣭⡇⠀⠀⢻⣿⣻⣿⣿⣿⣿⣿⡿⣳⣿⣿⣿⣿⣿⣿⣿⣿⡽⣿⣿⣿
⠼⠷⠸⠴⠀⠖⠆⠺⠇⠀⠸⠀⠧⠿⠿⢇⠀⠀⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣯
⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⢑⢔⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⢠⣦⣶⣔⣶⣦⣦⣶⣶⣶⣜⣿⣶⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⢸⣿⣿⣿⣻⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⢸⣿⣿⣿⣿⣾⣿⡽⣿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⢸⣿⣯⣿⣿⢿⣿⣿⣿⡟⣿⢿⣫⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠀⣸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠨⣸⣿⣿⣾⣿⣷⣾⡿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢤⢦⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢠⣼⢿⢿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢘⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣷⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢦⢹⡿⣻⡿⣿⣿⣿⣿⣿⣿⣾⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

In [21]:
mySourceFct(x,y,z) = x+y+z

function genLocVector(element::Element, sourceFct::Function)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4; 
    e1 = element.n1; e2 = element.n2; e3 = element.n3; e4 = element.n4; 
    vol = element.vol
    Iloc = SVector(e1, e2, e3, e4) 
    # use broadcast for the lines below instead 
    f1 = vol/6*sourceFct(p1[1],p1[2],p1[3])
    f2 = vol/6*sourceFct(p2[1],p2[2],p2[3])
    f3 = vol/6*sourceFct(p3[1],p3[2],p3[3])
    f4 = vol/6*sourceFct(p4[1],p4[2],p3[3])
    floc = SVector(f1, f2, f3, f4) 
    return Iloc, floc
end

function genVector(mesh, sourceFct::F) where F 
 
    #..recover number of elements  
    nelems  = mesh.nelements
    nnodes = mesh.nnodes 
     
    #..preallocate the memory for local matrix contributions 
    f = zeros(Float64,nnodes)

    for i = 1:nelems #..loop over number of elements..
        element::Element = mesh.Elements[i]
        Iloc, floc = genLocVector(element,sourceFct)
        f[Iloc] += floc 
    end
       
    return f; 
end

f = genVector(mesh, mySourceFct)

7350-element Vector{Float64}:
 0.0
 8.306614046595932e-6
 1.648457179068089e-5
 8.242285895353061e-6
 8.306614046657303e-6
 1.6613228093267654e-5
 2.4919842139976335e-5
 1.6613228093343023e-5
 3.8925537950083205e-7
 9.545748912260932e-7
 ⋮
 5.915761007380821e-5
 9.863046826166203e-5
 0.0002231207440567145
 0.00013293012869338203
 0.0001272144152948342
 0.00010688146881703048
 0.00012349166555379173
 0.00015333555733580448
 0.00012461453242690502

In [23]:
function handleBoundary(mesh,A,f)
    bndNodeIds = mesh.bndNodeIds; 
    #..handle essential boundary conditions 
    A[bndNodeIds,:] .= 0;
    A[bndNodeIds,bndNodeIds] = Diagonal(ones(size(bndNodeIds)))
    f[bndNodeIds] .= 0;
    return A, f  
end


A,f = handleBoundary(mesh,S,f)
u = A\f

7350-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.007711429330962998
 0.004600888186876024
 0.003050981898677189
 0.012299962626074116
 0.003269617889522714
 0.004327219026522261
 0.0029490129944934936
 0.004491950556739462
 0.004264283342038621